In [1]:
import pandas as pd

mnist = pd.read_csv('mnist.csv')

In [64]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, utils

num_epochs = 5 
num_classes = 10 
batch_size = 25
learning_rate = 0.001

In [65]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import gc

def create_dataset(df):
    X = df.drop('label', axis=1)
    X = MinMaxScaler().fit_transform(X)
    dataset = [[np.array([X[i].reshape(28, 28)]), df['label'].values[i]] for i in range(len(X))]
    del X
    gc.collect()
    return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [66]:
train_mnist, test_mnist = train_test_split(mnist)

train_loader = create_dataset(train_mnist)
test_loader = create_dataset(test_mnist)

c:\users\olga\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
c:\users\olga\appdata\local\programs\python\python36\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [67]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets
from bokeh.plotting import figure
from bokeh.io import show
from bokeh.models import LinearAxis, Range1d
import numpy as np

class ConvNet(nn.Module): 
    def __init__(self, activation_function): 
        super(ConvNet, self).__init__() 
        self.layer1 = nn.Sequential(nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1), 
                                    activation_function) 
        self.layer2 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1), 
                                    activation_function)
        self.layer3 = nn.Sequential(nn.Conv2d(8, 8, kernel_size=3, stride=1, padding=1), 
                                    activation_function)
        self.drop_out = nn.Dropout() 
        self.fc1 = nn.Linear(8 * 28 * 28, 64) 
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x): 
        out = self.layer1(x) 
        out = self.layer2(out) 
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1) 
        out = self.drop_out(out) 
        out = self.fc1(out) 
        out = self.fc2(out) 
        out = self.fc3(out)
        return out

In [90]:
def train(model, train_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_step = len(train_loader)
    loss_list = []
    acc_list = []
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_loader):
            outputs = model(images.float())
            loss = criterion(outputs, labels)
            loss_list.append(loss.item())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            acc_list.append(correct / total)
        print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                      .format(epoch + 1, num_epochs, loss.item(),
                              (correct / total) * 100))

In [91]:
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images.float())
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print('Test Accuracy of the model on the {} test images: {} %'.format(total, (correct / total) * 100))

In [92]:
digits_relu_model = ConvNet(nn.ReLU())
train(digits_relu_model, train_loader)

Epoch [1/5], Loss: 0.4000, Accuracy: 88.00%
Epoch [2/5], Loss: 0.1128, Accuracy: 92.00%
Epoch [3/5], Loss: 0.0262, Accuracy: 100.00%
Epoch [4/5], Loss: 0.3104, Accuracy: 88.00%
Epoch [5/5], Loss: 0.1708, Accuracy: 96.00%


In [93]:
test(digits_relu_model, test_loader)

Test Accuracy of the model on the 2500 test images: 96.12 %


In [94]:
digits_sigmoid_model = ConvNet(nn.Sigmoid())
train(digits_sigmoid_model, train_loader)

Epoch [1/5], Loss: 2.3071, Accuracy: 8.00%
Epoch [2/5], Loss: 2.2732, Accuracy: 12.00%
Epoch [3/5], Loss: 0.7781, Accuracy: 80.00%
Epoch [4/5], Loss: 0.4369, Accuracy: 88.00%
Epoch [5/5], Loss: 0.1824, Accuracy: 92.00%


In [95]:
test(digits_sigmoid_model, test_loader)

Test Accuracy of the model on the 2500 test images: 87.56 %


In [96]:
digits_tahn_model = ConvNet(nn.Tanh())
train(digits_tahn_model, train_loader)

Epoch [1/5], Loss: 0.5852, Accuracy: 88.00%
Epoch [2/5], Loss: 0.5344, Accuracy: 76.00%
Epoch [3/5], Loss: 0.2770, Accuracy: 88.00%
Epoch [4/5], Loss: 0.6808, Accuracy: 92.00%
Epoch [5/5], Loss: 0.1165, Accuracy: 92.00%


In [97]:
test(digits_tahn_model, test_loader)

Test Accuracy of the model on the 2500 test images: 89.92 %


In [83]:
import cv2
import os
import random
images = []
str_labels = 'ABCDEFGHIJ'
path = 'notMNIST_small'
for cur_dir, _, files in os.walk(path):
    for file in files:
        img = cv2.imread(os.path.join(cur_dir, file), 0)
        if img is None:
            continue
        label = str_labels.find(cur_dir[-1])
        assert label != -1
        images.append([np.array([img]), label])
random.shuffle(images)
letters_train_loader = DataLoader(dataset=images[:int(len(images) * 0.8)], batch_size=batch_size, shuffle=True) 
letters_test_loader = DataLoader(dataset=images[int(len(images) * 0.8):], batch_size=batch_size, shuffle=True) 

In [98]:
letters_relu_model = ConvNet(nn.ReLU())
train(letters_relu_model, letters_train_loader)

Epoch [1/5], Loss: 0.2802, Accuracy: 100.00%
Epoch [2/5], Loss: 0.0195, Accuracy: 100.00%
Epoch [3/5], Loss: 0.2244, Accuracy: 100.00%
Epoch [4/5], Loss: 0.0303, Accuracy: 100.00%
Epoch [5/5], Loss: 0.0535, Accuracy: 100.00%


In [99]:
test(letters_relu_model, letters_test_loader)

Test Accuracy of the model on the 3745 test images: 92.41655540720961 %


In [100]:
letters_sigmoid_model = ConvNet(nn.Sigmoid())
train(letters_sigmoid_model, letters_train_loader)

Epoch [1/5], Loss: 0.0627, Accuracy: 100.00%
Epoch [2/5], Loss: 0.4120, Accuracy: 75.00%
Epoch [3/5], Loss: 0.0308, Accuracy: 100.00%
Epoch [4/5], Loss: 1.3915, Accuracy: 75.00%
Epoch [5/5], Loss: 0.7033, Accuracy: 75.00%


In [101]:
test(letters_sigmoid_model, letters_test_loader)

Test Accuracy of the model on the 3745 test images: 90.41388518024031 %


In [102]:
letters_tahn_model = ConvNet(nn.Tanh())
train(letters_tahn_model, letters_train_loader)

Epoch [1/5], Loss: 0.0106, Accuracy: 100.00%
Epoch [2/5], Loss: 0.0791, Accuracy: 100.00%
Epoch [3/5], Loss: 0.0204, Accuracy: 100.00%
Epoch [4/5], Loss: 1.2357, Accuracy: 75.00%
Epoch [5/5], Loss: 1.6758, Accuracy: 75.00%


In [103]:
test(letters_tahn_model, letters_test_loader)

Test Accuracy of the model on the 3745 test images: 92.22963951935914 %
